# PEFT with LoRa

#### Steps:
1. Load a pre-trained model and evaluate its performance
2. Perform parameter-efficient fine-tuning using the pre-trained model
3. Perform inference using the fine-tuned model and compare its performance to the original model


In [1]:
!pip install transformers peft datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 25.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 40.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 37.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 790.9/790.9 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 33.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.1
    Uninstalling numpy-2.0.1:
      Successfully uninstalled numpy-2.0.1


In [8]:
# Imports
from sklearn.metrics import accuracy_score
import torch
from datasets import load_dataset
from transformers import Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from peft import LoraConfig, get_peft_model, AutoPeftModelForSequenceClassification



from transformers import DataCollatorWithPadding
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"


In [ ]:
# Define Model
DATASET_NAME: str = 'imdb' #'AdaptLLM/law_knowledge_prob'
MODEL_NAME: str = 'distilgpt2'

splits = ["train","test"]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()

In [ ]:
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# Assign the eos_token as the pad_token
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
# Load the datasets
ds =load_dataset(DATASET_NAME)#['train'].train_test_split(test_size=0.2,shuffle=True, seed=42)
print(ds)

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [ ]:
data = {}
for split in splits:
    data[split] = ds[split].map(lambda x: tokenizer(x['text'],padding=True, truncation=True),
            batched=True).shuffle(True).select(range(1000))

data

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

{'train': Dataset({
     features: ['text', 'label', 'input_ids', 'attention_mask'],
     num_rows: 1000
 }),
 'test': Dataset({
     features: ['text', 'label', 'input_ids', 'attention_mask'],
     num_rows: 1000
 })}

In [ ]:
# Load Pre-trained model

model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME,num_labels=2)
model.config.pad_token_id = tokenizer.pad_token_id  # Ensure the model is aware of the padding token
# Number of parameters in the model
total_params = model.num_parameters()
print(f"Total number of parameters: {total_params:,}")

print(model)

model.to(device)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at distilgpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Total number of parameters: 81,914,112
GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=2, bias=False)
)


GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=2, bias=False)
)

In [ ]:
def compute_metric(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    return {"accuracy": accuracy_score(labels, preds)}

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,
    per_device_eval_batch_size=32,
)

trainer = Trainer(
    model= model,
    args=training_args,
    train_dataset=data['train'],
    eval_dataset=data['test'],
    data_collator = DataCollatorWithPadding(tokenizer),
    compute_metrics=compute_metric,
)

# Evaluate the model on the validation set
print("Initial evaluation on the validation set:")
trainer.evaluate()

Initial evaluation on the validation set:


{'eval_loss': 0.7359223365783691,
 'eval_accuracy': 0.512,
 'eval_runtime': 35.5888,
 'eval_samples_per_second': 28.099,
 'eval_steps_per_second': 0.899}

In [ ]:

%md

# Fine Tune Model with LoRa PEFT Method

In [ ]:
# Perform peft using pre-trained model

from peft import LoraConfig

config = LoraConfig(
    r=4,
    lora_alpha=32,
    lora_dropout=0.1,
    fan_in_fan_out=True,
    bias='none',  # 'none', 'all', or 'lora_only'
    task_type='SEQ_CLS'  # Task type
)

lora_model = get_peft_model(model, config)

# Checking Trainable Parameters of a PEFT Model
lora_model.print_trainable_parameters()

trainable params: 75,264 || all params: 81,989,376 || trainable%: 0.0918


In [ ]:
# Training with a PEFT Model (PyTorch training loop or Hugging Face Trainer).


lora_trainer = Trainer(
    model= lora_model,
    args=training_args,
    train_dataset=data['train'],
    eval_dataset=data['test'],
    data_collator = DataCollatorWithPadding(tokenizer),
    compute_metrics=compute_metric,
)

lora_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.658910,0.619000
2,No log,0.648110,0.662000
3,No log,0.643708,0.672000


TrainOutput(global_step=189, training_loss=0.6706222049773686, metrics={'train_runtime': 416.5371, 'train_samples_per_second': 7.202, 'train_steps_per_second': 0.454, 'total_flos': 785305829376000.0, 'train_loss': 0.6706222049773686, 'epoch': 3.0})

Epoch,Training Loss,Validation Loss,Accuracy
0,No log,0.703415,0.515000
1,No log,0.703415,0.515000
2,No log,0.703415,0.515000


TrainOutput(global_step=45, training_loss=0.7005681779649523, metrics={'train_runtime': 4951.9239, 'train_samples_per_second': 0.606, 'train_steps_per_second': 0.009, 'total_flos': 753241298042880.0, 'train_loss': 0.7005681779649523, 'epoch': 2.88})

In [ ]:
# Evaluate the model on the validation set
print("Initial evaluation on the validation set:")
lora_trainer.evaluate()

Initial evaluation on the validation set:


{'eval_loss': 0.6437081098556519,
 'eval_accuracy': 0.672,
 'eval_runtime': 36.1316,
 'eval_samples_per_second': 27.677,
 'eval_steps_per_second': 0.886}

In [ ]:
# Saving a Trained PEFT Model

# Note that this only saves the adapter weights and not the weights of the original Transformers model.
# Thus the size of the files created will be much smaller than you might expect.
lora_model.save_pretrained("gpt_lora_mak")

# Inference with PEFT
# Loading a Saved PEFT Model
lora_model = AutoPeftModelForSequenceClassification.from_pretrained("gpt_lora_mak",num_labels=2)
lora_model.config.pad_token_id = tokenizer.pad_token_id  # Ensure the model is aware of the padding token

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at distilgpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
print(lora_model)

GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): lora.Linear(
            (base_layer): Conv1D()
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.1, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=768, out_features=4, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=4, out_features=2304, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
         

In [ ]:
# Evaluate the model on the validation set
print("Initial evaluation on the validation set:")
lora_trainer.evaluate()

Initial evaluation on the validation set:


{'eval_loss': 0.6437081098556519,
 'eval_accuracy': 0.672,
 'eval_runtime': 37.0392,
 'eval_samples_per_second': 26.998,
 'eval_steps_per_second': 0.864,
 'epoch': 3.0}